
### Software Design Enhancements

This enhanced version of the dashboard includes:
- Clearer project structure with renamed files
- Refactored layout naming for better maintainability
- Modular separation of logic and presentation layers
- Improved readability and comments throughout the code


<style>
    div.output_scroll {
    height: 1200px;
}
</style>

In [1]:
# Import all required libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pyplot as plt
import urllib.parse

# Import your CRUD module
from AnimalShelter import AnimalShelter



In [2]:
# Connect to MongoDB
username = urllib.parse.quote_plus("aacuser")
password = urllib.parse.quote_plus("Blathras")

shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read({}))

# Clean up the data
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [3]:
df.head()


rec_num age_upon_outcome animal_id animal_type                   breed  \
0      1.0          3 years   A746874         Cat  Domestic Shorthair Mix   
1     10.0         3 months   A664290         Cat  Domestic Shorthair Mix   
2     11.0           1 year   A721199         Dog  Dachshund Wirehair Mix   
3     12.0           1 year   A664843         Dog            Pit Bull Mix   
4     13.0           1 year   A700408         Cat  Domestic Shorthair Mix   

               color date_of_birth             datetime            monthyear  \
0        Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1             Tortie    2013-09-01  2013-12-08 14:58:00  2013-12-08T14:58:00   
2          Tan/White    2015-02-23  2016-02-27 17:49:00  2016-02-27T17:49:00   
3        Brown/White    2013-06-09  2014-08-18 17:24:00  2014-08-18T17:24:00   
4  Brown Tabby/White    2014-04-13  2015-04-15 13:34:00  2015-04-15T13:34:00   

       name outcome_subtype     outcome_type sex_upon_outcome  location_lat  \
0                      SCRP         Transfer    Neutered Male     30.506658   
1   *Taylor                         Adoption    Spayed Female     30.758311   
2     Belle                         Adoption    Spayed Female     30.729027   
3  Sherlock         Partner         Transfer    Neutered Male     30.451555   
4      Nyla                  Return to Owner    Spayed Female     30.410115   

   location_long  age_upon_outcome_in_weeks  
0     -97.340878                 156.767857  
1     -97.618292                  14.089087  
2     -97.375333                  52.820337  
3     -97.474105                  62.246429  
4     -97.562416                  52.509325

In [4]:
# Create Dash app layout
app = JupyterDash('ProjectTwoDashboard')

app.layout = html.Div([
    html.Img(src='assets/Glogo.png', style={'height': '100px'}),
    html.Center(html.H1("Project Two Dashboard - Steven Blathras")),

    html.H4("Rescue Type Filter"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'block'}
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Div(id='map-id', className='col s12 m6'),
    html.Br(),

    dcc.Graph(id='breed-chart'),
    html.Center(html.B("Steven Blathras - SNHU CS-340 MongoDB Dashboard"))
])

In [5]:
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children'),
     Output('breed-chart', 'figure')],
    [Input('rescue-type', 'value'),
     Input('datatable-id', 'selected_rows')]
)
def update_dashboard(rescue_type, selected_rows):
    dog_age_limit = 104  # weeks
    query = {"animal_type": "Dog", "age_upon_outcome_in_weeks": {"$lte": dog_age_limit}}

    rescue_breeds = {
        "Water": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
        "Mountain": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog"],
        "Disaster": ["Doberman Pinscher", "German Shepherd", "Golden Retriever"],
        "All": []
    }

    if rescue_type != "All":
        query["breed"] = {"$in": rescue_breeds[rescue_type]}

    results = shelter.read(query)
    dff = pd.DataFrame.from_records(results)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Handle empty dataset
    if dff.empty:
        return [], [html.P("No data available.")], {'data': [], 'layout': {'title': 'No data available'}}

    # Select row index
    row = selected_rows[0] if selected_rows else 0
    if row >= len(dff):
        row = 0

    # Map output
    map_output = [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[
                dff.iloc[row].get("location_lat", 30.75),
                dff.iloc[row].get("location_long", -97.48)],
                children=[
                    dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row].get("name", "Unknown"))
                    ])
                ])
        ])
    ]

    # Bar chart of breed counts
    breed_counts = dff['breed'].value_counts()
    chart_fig = {
        'data': [{
            'x': breed_counts.index,
            'y': breed_counts.values,
            'type': 'bar',
            'marker': {'color': 'teal'}
        }],
        'layout': {
            'title': 'Dog Breeds by Count (Filtered)',
            'xaxis': {'title': 'Breed'},
            'yaxis': {'title': 'Count'}
        }
    }

    return dff.to_dict('records'), map_output, chart_fig

In [6]:
# Run dashboard app in jupyter
app.run_server(mode='inline', debug=True)